In [ ]:
import urllib3
from urllib3 import request
import certifi
import json
import pandas as pd
import requests
import warnings

In [ ]:
#handling certification
http = urllib3.PoolManager(
    cert_reqs='CERT_REQUIRED',
    ca_certs = certifi.where()
)

In [ ]:
#calling api
# reg_api_key = 'LHWChdXGeFeoGXBjP5xEyGcqOoqAjetGXafZoX3r'
# reg_api_key = 'nZYc4Xpl1Gi4rJSIyZnlnK3MGOoiedgR6aPliIig'
reg_api_key = 'amji8EhA5cFJmGoBpP8dAgXoqaavhcApYcP3PTjD'
# docket_id = 'COLC-2023-0006'
docket_id = 'IRS-2023-0041'


def test_site(url):
  r = http.request('GET', url)
  if r.status == 200:
    print("Connection to the endpoint is good")
  else:
    print(f"there is an issue connecting to the endpoint. Error code: {r.status}")

def url2data(url):
  r = http.request('GET', url)
  loaded_data = json.loads(r.data.decode('utf-8'))
  return loaded_data

In [ ]:
#calling docket data
docket_url = f'https://api.regulations.gov/v4/documents?filter[docketId]={docket_id}&api_key={reg_api_key}'

test_site(docket_url)

#decoding the json data
docket_data = url2data(docket_url)
docket_data

Connection to the endpoint is good


{'data': [{'id': 'IRS-2023-0041-38875',
   'type': 'documents',
   'attributes': {'documentType': 'Proposed Rule',
    'lastModifiedDate': '2023-11-07T20:01:28Z',
    'highlightedContent': '',
    'frDocNum': '2023-24608',
    'withdrawn': False,
    'agencyId': 'IRS',
    'commentEndDate': None,
    'title': 'Gross Proceeds and Basis Reporting by Brokers and Determination of Amount Realized and Basis for Digital Asset Transactions',
    'postedDate': '2023-11-07T05:00:00Z',
    'docketId': 'IRS-2023-0041',
    'subtype': 'Public Hearing',
    'commentStartDate': '2023-11-07T05:00:00Z',
    'openForComment': False,
    'objectId': '0900006486218890'},
   'links': {'self': 'https://api.regulations.gov/v4/documents/IRS-2023-0041-38875'}},
  {'id': 'IRS-2023-0041-7617',
   'type': 'documents',
   'attributes': {'documentType': 'Proposed Rule',
    'lastModifiedDate': '2024-04-05T19:12:04Z',
    'highlightedContent': '',
    'frDocNum': '2023-23624',
    'withdrawn': False,
    'agencyId':

In [ ]:
#getting all document ids
document_ids = []
for item in docket_data['data']:
    if item['attributes']['commentStartDate'] is not None:
        document_id = item['attributes']['objectId']
        document_ids.append(document_id)

document_ids

['0900006486218890', '0900006486136cb5', '0900006485f0961d']

In [ ]:
#getting comments
document_id = document_ids[2]
document_url = f'https://api.regulations.gov/v4/comments?filter[commentOnId]={document_id}&page[size]=250&page[number]=1&sort=lastModifiedDate,documentId&api_key={reg_api_key}'
test_site(document_url)
document_data = url2data(document_url)
number_comments = document_data['meta']['totalElements']
print(f"there are {number_comments} comments.")

Connection to the endpoint is good
there are 44804 comments.


In [ ]:
#formatting data
def format_date(date):
  date = date.replace('T', ' ').replace('Z', '')
  return date
#getting json info
def url2data(url):
  r = http.request('GET', url)
  loaded_data = json.loads(r.data.decode('utf-8'))
  return loaded_data

In [ ]:
#getting last dates of the first 5k

#selecting page number
page_number = 20
#creating dataframe
df1 = pd.DataFrame(columns=['id', 'type',	'attributes.documentType', 'attributes.lastModifiedDate', 'attributes.highlightedContent',	'attributes.withdrawn',	'attributes.agencyId',	'attributes.title',	'attributes.objectId',	'attributes.postedDate',	'links.self'])
#creating string for data
comments_url= f'https://api.regulations.gov/v4/comments?filter[commentOnId]={document_id}&page[size]=250&page[number]={page_number}&sort=lastModifiedDate,documentId&api_key={reg_api_key}'
document_data = url2data(comments_url)
#creating dataframe with data
df2 = pd.json_normalize(document_data, 'data')
df1 = pd.concat([df1, df2], ignore_index=True)
last_mod_date = df1.iloc[-1]['attributes.lastModifiedDate']
first_date = df1.iloc[0]['attributes.lastModifiedDate']
last_date = format_date(last_mod_date)

print(first_date)
print(last_date)


2023-10-24T17:52:20Z
2023-10-24 17:56:02


In [ ]:
count = 5000

In [ ]:
#nextdocuments
print(last_date)
#selecting page number
page_number = 1
#creating dataframe
df1 = pd.DataFrame(columns=['id', 'type',	'attributes.documentType', 'attributes.lastModifiedDate', 'attributes.highlightedContent',	'attributes.withdrawn',	'attributes.agencyId',	'attributes.title',	'attributes.objectId',	'attributes.postedDate',	'links.self'])
comments_url= f'https://api.regulations.gov/v4/comments?sort=lastModifiedDate,documentId&filter[commentOnId]={document_id}&filter[lastModifiedDate][ge]={last_date}&page[size]=250&page[number]={page_number}&api_key={reg_api_key}'
document_data = url2data(comments_url)
print(document_data)
#creating dataframe with data
df2 = pd.json_normalize(document_data, 'data')
count += len(df2)
df1 = pd.concat([df1, df2], ignore_index=True)
last_mod_date = df1.iloc[-1]['attributes.lastModifiedDate']
first_date = df1.iloc[0]['attributes.lastModifiedDate']
last_date = format_date(last_mod_date)
print(first_date)
print(last_date)
# print(count)
# df1

2023-10-24 17:56:02
{'data': [{'id': 'IRS-2023-0041-7618', 'type': 'comments', 'attributes': {'documentType': 'Public Submission', 'lastModifiedDate': '2023-10-25T16:06:42Z', 'highlightedContent': '', 'withdrawn': False, 'agencyId': 'IRS', 'title': 'Comment from Tannenbaum, Jesse', 'objectId': '09000064861231f6', 'postedDate': '2023-10-25T04:00:00Z'}, 'links': {'self': 'https://api.regulations.gov/v4/comments/IRS-2023-0041-7618'}}, {'id': 'IRS-2023-0041-7619', 'type': 'comments', 'attributes': {'documentType': 'Public Submission', 'lastModifiedDate': '2023-10-25T16:57:20Z', 'highlightedContent': '', 'withdrawn': False, 'agencyId': 'IRS', 'title': 'Comment from Tee, David', 'objectId': '09000064861231fa', 'postedDate': '2023-10-25T04:00:00Z'}, 'links': {'self': 'https://api.regulations.gov/v4/comments/IRS-2023-0041-7619'}}, {'id': 'IRS-2023-0041-7620', 'type': 'comments', 'attributes': {'documentType': 'Public Submission', 'lastModifiedDate': '2023-10-25T16:57:21Z', 'highlightedContent'

In [ ]:
document_data

{'data': [{'id': 'IRS-2023-0041-14239',
   'type': 'comments',
   'attributes': {'documentType': 'Public Submission',
    'lastModifiedDate': '2023-10-31T19:28:10Z',
    'highlightedContent': '',
    'withdrawn': False,
    'agencyId': 'IRS',
    'title': 'Comment from Graham, Pamela',
    'objectId': '090000648613534e',
    'postedDate': '2023-10-31T04:00:00Z'},
   'links': {'self': 'https://api.regulations.gov/v4/comments/IRS-2023-0041-14239'}},
  {'id': 'IRS-2023-0041-14240',
   'type': 'comments',
   'attributes': {'documentType': 'Public Submission',
    'lastModifiedDate': '2023-10-31T19:28:11Z',
    'highlightedContent': '',
    'withdrawn': False,
    'agencyId': 'IRS',
    'title': 'Comment from Crase, Heidi',
    'objectId': '090000648613534f',
    'postedDate': '2023-10-31T04:00:00Z'},
   'links': {'self': 'https://api.regulations.gov/v4/comments/IRS-2023-0041-14240'}},
  {'id': 'IRS-2023-0041-14241',
   'type': 'comments',
   'attributes': {'documentType': 'Public Submissio

In [ ]:
adobe = f'https://api.regulations.gov/v4/comments/COLC-2023-0006-8594?include=attachments&api_key={reg_api_key}'
adobe_data = url2data(adobe)
adobe_data

{'data': {'id': 'COLC-2023-0006-8594',
  'type': 'comments',
  'links': {'self': 'https://api.regulations.gov/v4/comments/COLC-2023-0006-8594'},
  'attributes': {'commentOn': '0900006485f112d2',
   'commentOnDocumentId': 'COLC-2023-0006-0001',
   'duplicateComments': 1,
   'address1': None,
   'address2': None,
   'agencyId': 'COLC',
   'city': None,
   'category': None,
   'comment': 'See attached file(s)',
   'country': None,
   'displayProperties': None,
   'docAbstract': None,
   'docketId': 'COLC-2023-0006',
   'documentType': 'Public Submission',
   'email': None,
   'fax': None,
   'field1': None,
   'field2': None,
   'fileFormats': None,
   'firstName': None,
   'govAgency': None,
   'govAgencyType': None,
   'objectId': '0900006486186ab3',
   'lastName': None,
   'legacyId': None,
   'modifyDate': '2023-11-01T17:56:16Z',
   'organization': 'Adobe Inc.',
   'originalDocumentId': None,
   'pageCount': 0,
   'phone': None,
   'postedDate': '2023-11-01T04:00:00Z',
   'postmarkDat